In [106]:
import pandas as pd
import os
import glob
 
# use glob to get all the csv files in the folder
dict = {}
path = r"C:\Users\anton\OneDrive\Documentos\Georgian\BDAT1007_Social_Data_Mining\MidTerm"
csv_files = glob.glob(os.path.join(path, "*.csv"))

for f in csv_files:
    #Split the pat and get only the filename without extension
    file_name = os.path.basename(f)
    file = os.path.splitext(file_name)
    # read the csv file and store in the dictionary
    dict[file[0]] = pd.read_csv(f)

In [107]:
#Saving all the files in df
book_tags = pd.DataFrame.from_dict(dict.get("book_tags"))
books = pd.DataFrame.from_dict(dict.get("books"))
ratings = pd.DataFrame.from_dict(dict.get("ratings"))
tags = pd.DataFrame.from_dict(dict.get("tags"))
to_read = pd.DataFrame.from_dict(dict.get("to_read"))

In [3]:
from sklearn import *
# TF-IDF Feature Generation
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer

# Initialize regex tokenizer
tokenizer = RegexpTokenizer(r'\w+')

# # Vectorize document using TF-IDF
tf_idf_vect = TfidfVectorizer(lowercase=True,
                        stop_words='english',
                        ngram_range = (1,1),
                        tokenizer = tokenizer.tokenize)

# Fit and Transfrom Text Data
X_train_counts = tf_idf_vect.fit_transform(tags["tag_name"])

# Check Shape of Count Vector
X_train_counts.shape

(34252, 16450)

In [46]:
# Import KMeans Model
from sklearn.cluster import KMeans

# Create Kmeans object and fit it to the training data 
kmeans = KMeans(n_clusters=20).fit(X_train_counts)

# Get the labels using KMeans
pred_labels = kmeans.labels_

In [25]:
from sklearn import metrics
# Compute DBI score
dbi = metrics.davies_bouldin_score(X_train_counts.toarray(), pred_labels)

# Compute Silhoutte Score
ss = metrics.silhouette_score(X_train_counts.toarray(), pred_labels , metric='euclidean')

# Print the DBI and Silhoutte Scores
print("DBI Score: ", dbi, "\nSilhoutte Score: ", ss)

DBI Score:  2.9302840334103455 
Silhoutte Score:  0.03769045111809106


In [47]:
pred_labels

array([0, 0, 0, ..., 0, 0, 0])

In [75]:
tags["group"] = pred_labels
#dataframe[dataframe['Percentage'] > 70]
tags[tags["group"] ==1]
#the result categories are:
#variety, manga, fiction, women, read books, arab, books, fantasy, middle, kids, children, read, book, james, hyphen, 2016, romance, Stephen King, -s, related

,tag_id,tag_name,group
165,165,1-manga-manhwa-manhua-series-done,1
873,873,2017-manga,1
1547,1547,action-manga,1
2595,2595,anime-manga,1
2991,2991,art-comics-manga,1
...,...,...,...
31737,31737,vampire-knight-manga,1
32114,32114,want-to-read-manga,1
32598,32598,wishlist-manga,1
32952,32952,x03_all-manga-comic__1123_,1


In [108]:
#To improve thhe results we are going to remove the hyphen symbol in the tags df
tags["tag_name"] = tags["tag_name"].str.replace('-', ' ')
#removing alone characters to improve the results

tags["tag_name"] = tags["tag_name"].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
tags.head()

C:\Users\anton\AppData\Local\Temp/ipykernel_21380/132396117.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  tags["tag_name"] = tags["tag_name"].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')


,tag_id,tag_name
0,0,
1,1,
2,2,10
3,3,12
4,4,122


In [109]:
tokenizer = RegexpTokenizer(r'\w+')

# # Vectorize document using TF-IDF
tf_idf_vect = TfidfVectorizer(lowercase=True,
                        stop_words='english',
                        ngram_range = (1,1),
                        tokenizer = tokenizer.tokenize)

# Fit and Transfrom Text Data
X_train_counts = tf_idf_vect.fit_transform(tags["tag_name"])

# Check Shape of Count Vector
X_train_counts.shape

# Create Kmeans object and fit it to the training data 
kmeans = KMeans(n_clusters=20).fit(X_train_counts)

# Get the labels using KMeans
pred_labels = kmeans.labels_


tags["group"] = pred_labels

In [145]:
tags[tags["group"] ==13]
#The new groups are:
#various, romance, school, best books, other books, reading, read, learning, lit, series, novels, summer, brown, philosophy, owned, fiction
# wish, game, kids, reads
tags[

,tag_id,tag_name,group
508,508,19th philosophy,13
2408,2408,analytic philosophy,13
2409,2409,analytical philosophy,13
2433,2433,ancient greek philosophy,13
2444,2444,ancient philosophy,13
...,...,...,...
30243,30243,theology and philosophy,13
30251,30251,theology philosophy,13
30261,30261,theory philosophy,13
30776,30776,to read philosophy,13
